# Cheat sheet de estadística para ciencia de datos

## Probabilidad 

### Teorema de Bayes

#### Probabilidad condicional

##### Ejemplos típicos

## Distribuciones típicas

### Discretas

### Continuas


## Análisis de significancia

#### ANOVA:

##### ANOVA: Efectos fijos

##### ANOVA: Efectos aleatorios

#### Análisis de la covarianza (ANCOVA):


## Tests clásicos

#### AB Testing

#### Test de medias

#### Test $\chi^2$

#### Test $t$-student

## Test no paramétricos:

##### Kolmogorov Smirnoff

##### Basados en momentos:

###### Dagostino Pearson

###### Jarque Bera

#### Kruskal-Wallis

#### Friedman

#### Test de rachas de Wolfowitz

#### Test de wilcoxon

####